# 訓練、評価、テストデータ作成

In [11]:
#pathの追加（必要に応じて）
import sys
#sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python36.zip')
#sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6')
#sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6/lib-dynload')
#sys.path.append('/home/siny/.local/lib/python3.6/site-packages')
#sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6/site-packages')

In [28]:
import csv
import random


#import pandas as pd
#from pathlib import Path
#import json

## Twitter日本語評判分析データセット取得

In [54]:
data_path = "data/Twitter日本語評判分析データセット.txt"
nega_data = []
posi_data = []
neut_data = []
nandp_data = []
with open(data_path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"')
    head = next(reader) # ヘッダ行は不要
    for row in reader:
        tweet = row[4]
        tweet = tweet.replace("\n","") # 改行は除く
        if row[3] == 'positive':
            posi_data.append(tweet)
        elif row[3] == 'negative':
            nega_data.append(tweet)
        elif row[3] == 'neutral':
            neut_data.append(tweet)
        elif row[3] == 'NegaAndPosi':
            nandp_data.append(tweet)
        else:
            print('予期しない感情がありました：' + row[3])
            break
print('ネガティブ数：{}、ポジティブ数：{}、ニュートラル数：{}、ネガ＆ポジ数：{}'.format(
    len(nega_data),len(posi_data),len(neut_data),len(nandp_data)))

ネガティブ数：1427、ポジティブ数：823、ニュートラル数：7121、ネガ＆ポジ数：53


## 訓練・検証・テストデータに分割
- 訓練・検証データに含める感情はネガティブ、ポジティブ。
- テストデータに含める感情は、ネガティブ、ポジティブ、ニュートラル、ネガ＆ポジ
- train.tsv：ネガティブ、ポジティブそれぞれ60%。
- validate.tsv：ネガティブ、ポジティブそれぞれ20%。
- test.tsv：ネガティブ、ポジティブそれぞれ20%。ニュートラル、ネガ＆ポジ。

In [60]:
random.shuffle(nega_data)
random.shuffle(posi_data)

threshold1 = 0.6
threshold2 = 0.8
neg_num = len(nega_data)
pos_num = len(posi_data)


In [61]:
#ネガポジデータの生成(訓練データ)
with open("data/train.tsv", "a", encoding='utf-8') as f:
    index = 0
    for text in nega_data:
        if index < neg_num * threshold1:
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            break
    index = 0
    for text in posi_data:
        if index < pos_num * threshold1:
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            break

In [62]:
# 検証データ作成
with open("data/test.tsv", "a", encoding='utf-8') as f:
    index = 0
    for text in nega_data:
        if index >= neg_num * threshold1 and index < neg_num * threshold2:
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            index += 1
    index = 0
    for text in posi_data:
        if index >= pos_num * threshold1 and index < pos_num * threshold2:
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            index += 1

In [63]:
# テストデータ（ネガティブ、ポジティブ）作成
with open("data/test2.tsv", "a", encoding='utf-8') as f:
    index = 0
    for text in nega_data:
        if index >= neg_num * threshold2:
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            index += 1
    index = 0
    for text in posi_data:
        if index >= pos_num * threshold2:
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            index += 1

In [64]:
# テストデータ（ニュートラル、ネガ＆ポジ）作成
with open("data/test2_other.tsv", "a", encoding='utf-8') as f:
    for text in neut_data:
        text = text+'\t'+'2'+'\t'+'\n'
        f.write(text)
    for text in nandp_data:
        text = text+'\t'+'3'+'\t'+'\n'
        f.write(text)